In [1]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [1]:
# 2D Rosenbrock function
def f(x):
    return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

In [3]:
# 2D Rastrigin function
def f(x):
    return -(20 + x[0]**2 + x[1]**2 - 10*(torch.cos(2*pi*x[0])+torch.cos(2*pi*x[1])))

In [3]:
# N-D Rosenbrock function
def f(x):
    rosen=0
    for i in range(x.numel()-1):
        rosen += 100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2
        print(f"100*(x[{i+1}] - x[{i}]**2)**2 + (1 - x[{i}])**2")
    return rosen

In [5]:
import torch
import torch._dynamo
import os
torch._dynamo.reset()
# torch._dynamo.explain(f, torch.rand(2))

os.environ['TORCH_COMPILE_DEBUG']='1'

t = torch.rand(5)
cf = torch.compile(f, options={'trace.graph_diagram':True,
                                'trace.enabled':True})
cf(t)

[2023-03-22 06:37:10,870] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing f
[2023-03-22 06:37:10,871] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_465/783128143.py:3
[2023-03-22 06:37:10,871] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_CONST 0 []
[2023-03-22 06:37:10,872] torch._dynamo.symbolic_convert: [DEBUG] TRACE STORE_FAST rosen [ConstantVariable(int)]
[2023-03-22 06:37:10,872] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_465/783128143.py:4
[2023-03-22 06:37:10,872] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_GLOBAL range []
[2023-03-22 06:37:10,873] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x [BuiltinVariable(range)]
[2023-03-22 06:37:10,873] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR numel [BuiltinVariable(range), TensorVariable()]
[2023-03-22 06:37:10,874] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 0 [BuiltinVariable(range), GetAttrVariable(

100*(x[1] - x[0]**2)**2 + (1 - x[0])**2
100*(x[2] - x[1]**2)**2 + (1 - x[1])**2
100*(x[3] - x[2]**2)**2 + (1 - x[2])**2
100*(x[4] - x[3]**2)**2 + (1 - x[3])**2


tensor(69.7175)

In [3]:
def f(x):
    rosen=0
    for i in range(x.size(dim=0)-1):
        rosen += 100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2
        print(f"100*(x[{i}+1] - x[{i}]**2)**2 + (1 - x[{i}])**2")
    return rosen

In [ ]:
import torch
import math
import torch.optim as optim
import matplotlib.pyplot as plt
pi = torch.tensor(math.pi)
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [ ]:
# L-BFGS
import random
torch.manual_seed(0)
random.seed(0)

def closure():
    optimizer.zero_grad()
    f_eval = f(x_i)
    f_eval.backward()
    return f_eval

x_i = torch.randn(100,requires_grad=True, device="cpu")
print(f(x_i))
optimizer = optim.LBFGS([x_i],
                        lr=1,
                        max_iter=500, 
                        line_search_fn="strong_wolfe")

In [ ]:
import torch._dynamo
torch._dynamo.config.verbose=True
f = torch.compile(f,options={'trace.graph_diagram':True,
                                'trace.enabled':True})
f_eval_prev = f(x_i)

In [ ]:
%time optimizer.step(closure)
f_eval = f(x_i)
print(f"x:{x_i}, fn_prev:{f_eval_prev} fn:{f_eval}")

In [ ]:
%%time
counter = 0
while (f_eval_prev-f_eval) > 1e-10:
    counter+=1
    f_eval_prev = f_eval
    counter+=1
    optimizer.step(closure)
    f_eval = f(x_i)
    if not counter%1:
        print(f"x:{x_i}, fn:{f_eval}")
print(f"Steps: {counter}")
print(f"x:{x_i}, fn_prev:{f_eval_prev} fn:{f_eval}")

Steps: 344684

x:tensor([0.9989, 0.9979], requires_grad=True), fn_prev:1.1128381629532669e-06 fn:1.112765403377125e-06

CPU times: user 46.5 s, sys: 42.8 ms, total: 46.5 s

Wall time: 46.4 s

In [ ]:
f_eval = f(x_i)
while f_eval > 1e-5:
    counter+=1
    f_eval = f(x_i)
    optimizer.step(closure)

print(x_i)
print(f(x_i))
optimizer.defaults